In [ ]:
# from pathlib import Path
# import json
# import pandas as pd
# import math
# import json
# import os
# from typing import Dict, List
# import json
# from collections import defaultdict
# from itertools import combinations
# import os

# # ====== 固定处理这个 path ======
# BASE_PATH = Path(
#     "/data/userdata/v-lijingyuan/JobAndExp/amlt_project/logs_allowed-parrot/20260108-005156_aerial-cactus-identification"
# )

# records = []


# def collect_metric_steps(step, step_to_data, visited=None):
#     if visited is None:
#         visited = set()

#     if step in visited:
#         return []

#     visited.add(step)

#     data = step_to_data.get(step)
#     if data is None:
#         return []

#     results = []

#     for p in data.get("parents", []):
#         results.extend(collect_metric_steps(p, step_to_data, visited))

#         parent_data = step_to_data.get(p)
#         if parent_data is not None and parent_data.get("metric") is not None:
#             results.append(p)

#     return results


# def build_parent_plan_chain(step, step_to_data):
#     steps = collect_metric_steps(step, step_to_data)

#     self_data = step_to_data.get(step)
#     if self_data is not None and self_data.get("metric") is not None:
#         steps.append(step)

#     if not steps:
#         return "ROOT"

#     steps = sorted(set(steps))

#     return " -> ".join(
#         step_to_data[s].get("plan", "") for s in steps
#     )


# def find_nearest_parent_metric(step, step_to_data, visited=None):
#     """
#     找最近的一个有 metric 的 parent
#     """
#     if visited is None:
#         visited = set()
#     if step in visited:
#         return None

#     visited.add(step)

#     data = step_to_data.get(step)
#     if data is None:
#         return None

#     for p in data.get("parents", []):
#         parent_data = step_to_data.get(p)
#         if parent_data is None:
#             continue

#         if parent_data.get("metric") is not None:
#             return parent_data.get("metric")

#         hit = find_nearest_parent_metric(p, step_to_data, visited)
#         if hit is not None:
#             return hit

#     return None



# for journal_path in BASE_PATH.rglob("json/JOURNAL.jsonl"):
#     competition = BASE_PATH.name.split("_", 1)[-1]
#     experiment = "allowed-parrot"

#     # ---------- Phase 1 ----------
#     step_to_data = {}
#     rows = []

#     with open(journal_path) as f:
#         for line in f:
#             if not line.strip():
#                 continue
#             obj = json.loads(line)
#             data = obj["data"]
#             rows.append(data)

#             step = data.get("step")
#             if step is not None:
#                 step_to_data[step] = data

#     # ---------- Phase 2 ----------
#     for data in rows:
#         self_metric = data.get("metric")
#         parent_metric = find_nearest_parent_metric(
#             data.get("step"),
#             step_to_data
#         )

#         if self_metric is not None and parent_metric is not None:
#             score_diff = self_metric - parent_metric
#         else:
#             score_diff = float("nan")

#         records.append({
#             "competition": competition,
#             "experiment": experiment,
#             "plan": data.get("plan"),
#             "metric": self_metric,
#             "metric_maximize": data.get("metric_maximize"),
#             "hypothesis_chain": build_parent_plan_chain(
#                 data.get("step"),
#                 step_to_data
#             ),
#             "score diff": score_diff,
#         })

# df = pd.DataFrame(records)


In [ ]:
# from itertools import combinations
# import pandas as pd
# import json

# json_records = []

# # 只保留有 score diff 的
# df_valid = df[df["score diff"].notna()].copy()

# for (competition, experiment), g in df_valid.groupby(["competition", "experiment"]):
#     items = g.to_dict("records")
#     if len(items) < 2:
#         continue

#     # 是否越大越好
#     bigger_is_better = bool(items[0]["metric_maximize"])

#     # 统一方向：maximize -> +1, minimize -> -1
#     direction = 1 if bigger_is_better else -1

#     for a, b in combinations(items, 2):
#         score_a = direction * a["score diff"]
#         score_b = direction * b["score diff"]

#         # 判定 winner / loser
#         if score_a == score_b:
#             continue  # 可选：完全一样就跳过

#         winner, loser = (a, b) if score_a > score_b else (b, a)

#         # pair 的相对优势强度（始终 >= 0）
#         pair_score_diff = abs(winner["score diff"] - loser["score diff"])

#         json_records.append({
#             "competition": competition,
#             "winner": winner["hypothesis_chain"],
#             "loser": loser["hypothesis_chain"],
#             "score_diff": pair_score_diff,
#         })

# # 写 JSON
# with open("hypothesis_pairs.json", "w", encoding="utf-8") as f:
#     json.dump(json_records, f, indent=2, ensure_ascii=False)

# print(f"Total json records: {len(json_records)}")
# print(json_records[0])


In [ ]:
# json_records

In [ ]:
from pathlib import Path
import json
import pandas as pd
from itertools import combinations
    #"/data/userdata/v-lijingyuan/JobAndExp/amlt_project/logs_social-crawdad"
    #"/data/userdata/v-lijingyuan/JobAndExp/logs_useful-egret"
# ====== 所有 competition 的父目录 ======
ROOT_PATH = Path(
    #"/data/userdata/v-lijingyuan/JobAndExp/logs_useful-egret"
    #"/data/userdata/v-lijingyuan/JobAndExp/logs_relaxing-cowbird"
    "/data/userdata/v-lijingyuan/JobAndExp/amlt_project/logs_cute-mammoth"
    #"/data/userdata/v-lijingyuan/JobAndExp/amlt_project/logs_relaxing-cowbird"
)

json_records = []


def collect_metric_steps(step, step_to_data, visited=None):
    if visited is None:
        visited = set()
    if step in visited:
        return []
    visited.add(step)

    data = step_to_data.get(step)
    if data is None:
        return []

    results = []
    for p in data.get("parents", []):
        results.extend(collect_metric_steps(p, step_to_data, visited))
        parent_data = step_to_data.get(p)
        if parent_data is not None and parent_data.get("metric") is not None:
            results.append(p)
    return results


def build_parent_plan_chain(step, step_to_data):
    steps = collect_metric_steps(step, step_to_data)

    self_data = step_to_data.get(step)
    if self_data is not None and self_data.get("metric") is not None:
        steps.append(step)

    if not steps:
        return "ROOT"

    steps = sorted(set(steps))
    return "->".join(step_to_data[s].get("plan", "") for s in steps)


def find_nearest_parent_metric(step, step_to_data, visited=None):
    if visited is None:
        visited = set()
    if step in visited:
        return None
    visited.add(step)

    data = step_to_data.get(step)
    if data is None:
        return None

    for p in data.get("parents", []):
        parent_data = step_to_data.get(p)
        if parent_data is None:
            continue
        if parent_data.get("metric") is not None:
            return parent_data.get("metric")
        hit = find_nearest_parent_metric(p, step_to_data, visited)
        if hit is not None:
            return hit
    return None


# ============================================================
# 遍历：competition → experiment → JOURNAL.jsonl
# ============================================================
for competition_dir in ROOT_PATH.iterdir():
    if not competition_dir.is_dir():
        continue

    # competition 名
    competition = competition_dir.name.split("_", 1)[-1]

    for journal_path in competition_dir.rglob("json/JOURNAL.jsonl"):
        experiment = competition_dir.name.split("_", 1)[0].replace("logs_", "")
        # ---------- Phase 1 ----------
        step_to_data = {}
        rows = []

        with open(journal_path) as f:
            for line in f:
                if not line.strip():
                    continue
                obj = json.loads(line)
                data = obj["data"]
                rows.append(data)
                step = data.get("step")
                if step is not None:
                    step_to_data[step] = data

        # ---------- Phase 2：构造 df ----------
        records = []
        for data in rows:
            #print(data)
            self_metric = data.get("metric")
            parent_metric = find_nearest_parent_metric(
                data.get("step"), step_to_data
            )
            #print(self_metric,parent_metric)
            
            if self_metric is not None and parent_metric is not None:
                score_diff = self_metric - parent_metric
            else:
                score_diff = float("nan")

            records.append({
                "competition": competition,
                "experiment": experiment,
                "metric_maximize": data.get("metric_maximize"),
                "hypothesis_chain": build_parent_plan_chain(
                    data.get("step"), step_to_data
                ),
                "score_diff": score_diff,
            })
        #print(len(records))
        df = pd.DataFrame(records)
        df_valid = df[df["score_diff"].notna()]
        #print(len(df_valid))
        if len(df_valid) < 2:
            continue
        #print(df_valid)
        # ---------- Phase 3：pairwise preference ----------
        items = df_valid.to_dict("records")
        bigger_is_better = bool(items[0]["metric_maximize"])
        direction = 1 if bigger_is_better else -1

        for a, b in combinations(items, 2):
            score_a = direction * a["score_diff"]
            score_b = direction * b["score_diff"]

            if score_a == score_b:
                continue

            winner, loser = (a, b) if score_a > score_b else (b, a)
            pair_score_diff = abs(winner["score_diff"] - loser["score_diff"])
            #print(competition)
            json_records.append({
                "competition": competition,
                "winner": winner["hypothesis_chain"],
                "loser": loser["hypothesis_chain"],
                "score_diff": pair_score_diff,
            })


# ===============================
# 输出 JSON
# ===============================
with open("/data/userdata/v-lijingyuan/vail_data/aira1.json", "w", encoding="utf-8") as f:
    json.dump(json_records, f, indent=2, ensure_ascii=False)

print(f"Total preference pairs: {len(json_records)}")
#print(json_records[0])


In [ ]:
import json

h1_path = "/data/userdata/v-lijingyuan/vail_data/h1.json"
h2_path = "/data/userdata/v-lijingyuan/vail_data/h2.json"
h3_path = "/data/userdata/v-lijingyuan/vail_data/h3.json"

output_path = "/data/userdata/v-lijingyuan/vail_data/valid.json"

# 读取 H1 和 H2
with open(h1_path, "r") as f:
    h1_data = json.load(f)

with open(h2_path, "r") as f:
    h2_data = json.load(f)
with open(h3_path, "r") as f:
    h3_data = json.load(f)
# 简单合并两个列表
combined = h1_data + h2_data+h3_data

# # 保存
# with open(output_path, "w") as f:
#     json.dump(combined, f, indent=2)

# print(f"Combined JSON saved to {output_path}")


In [ ]:
def build_pairs_from_journal(path: str):
    """
    从单个 journal.json 构造 preference pairs
    """
    # -------- 1. extract competition name --------
    base = os.path.basename(os.path.dirname(os.path.dirname(path)))
    # e.g. text-normalization-challenge-english-language_xxxxx
    comp_name = base.split("_")[0]

    # -------- 2. load data --------
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    nodes = data["nodes"]
    node2parent = data.get("node2parent", {})
    node_dict = {n["id"]: n for n in nodes}

    # -------- 3. helpers --------
    def get_score(node):
        if node is None:
            return 0.0
        val = node.get("metric", {}).get("value")
        return 0.0 if val is None else float(val)

    def get_reward(node, parent_node):
        metric = node.get("metric", {})
        maximize = metric.get("maximize")

        cur_score = get_score(node)
        parent_score = get_score(parent_node)

        if maximize is True:
            return cur_score - parent_score
        elif maximize is False:
            return parent_score - cur_score
        else:
            # maximize is None
            return 0.0

    def build_plan_chain_from_root(node_id: str) -> str:
        chain = []
        cur = node_id
        while cur is not None:
            plan = node_dict[cur].get("plan")
            if plan is not None:
                chain.append(plan)
            cur = node2parent.get(cur)
        return " -> ".join(reversed(chain))

    # -------- 4. build node-level results --------
    results = []

    for node in nodes:
        node_id = node["id"]
        parent_id = node2parent.get(node_id)
        parent_node = node_dict[parent_id] if parent_id is not None else None

        cur_score = get_score(node)
        parent_score = get_score(parent_node)

        results.append({
            "node_id": node_id,
            "plan_chain": build_plan_chain_from_root(node_id),
            "cur_score": cur_score,
            "parent_score": parent_score,
            "score_diff": cur_score - parent_score,
            "reward": get_reward(node, parent_node),
            "maximize": node.get("metric", {}).get("maximize"),
        })

    # -------- 5. group by parent & build pairs --------
    parent2children = defaultdict(list)
    for r in results:
        parent_id = node2parent.get(r["node_id"])
        if parent_id is not None:
            parent2children[parent_id].append(r)

    pairs = []

    for parent_id, children in parent2children.items():
        if len(children) < 2:
            continue

        for a, b in combinations(children, 2):
            if a["reward"] == b["reward"]:
                continue

            if a["reward"] > b["reward"]:
                better, worse = a, b
            else:
                better, worse = b, a

            pairs.append({
                "agent":"aide",
                "comptation_name": comp_name,
                "parent_id": parent_id,
                #"better_node_id": better["node_id"],
                #"worse_node_id": worse["node_id"],
                "winner": better["plan_chain"],
                "loser": worse["plan_chain"],
                "score_diff": better["reward"] - worse["reward"],
            })

    return pairs

import os


import os
import random
from collections import defaultdict


def build_pairs_from_run_group(
    run_group_dir: str,
    max_pairs_per_comp: int = 500,
    seed: int = 42,
):
    """
    遍历 run-group 目录下所有比赛，汇总 preference pairs
    对每个 competition 施加 pair 数量上限
    """
    random.seed(seed)

    comp2pairs = defaultdict(list)
    missing = []

    for name in os.listdir(run_group_dir):
        comp_dir = os.path.join(run_group_dir, name)
        if not os.path.isdir(comp_dir):
            continue

        journal_path = os.path.join(comp_dir, "logs", "journal.json")
        if not os.path.isfile(journal_path):
            missing.append(comp_dir)
            continue

        try:
            pairs = build_pairs_from_journal(journal_path)

            # competition 名从目录中抽
            comp_name = name.split("_")[0]
            comp2pairs[comp_name].extend(pairs)

            print(f"[OK] {name}: {len(pairs)} pairs")
        except Exception as e:
            print(f"[ERROR] {name}: {e}")

    # -------- apply per-competition cap --------
    all_pairs = []

    for comp, plist in comp2pairs.items():
        original = len(plist)

        if original > max_pairs_per_comp:
            plist = random.sample(plist, max_pairs_per_comp)
            print(f"[CAP] {comp}: {original} -> {max_pairs_per_comp}")

        all_pairs.extend(plist)

    print("=" * 80)
    print(f"Total competitions processed: {len(comp2pairs)}")
    print(f"Total preference pairs: {len(all_pairs)}")
    print(f"Missing journal.json: {len(missing)}")

    return all_pairs


In [ ]:
import pandas as pd

df = pd.DataFrame(combined)

# 需要限流的 competition
CAP_COMPS = {
    "random-acts-of-pizza": 2000,
    "text-normalization-challenge-russian-language": 2000,
    "nomad2018-predict-transparent-conductors":2000,
    "detecting-insults-in-social-commentary":2000,
    "mlsp-2013-birds":2000,
    "new-york-city-taxi-fare-prediction":2000
}

dfs = []

for comp, g in df.groupby("competition"):
    if comp in CAP_COMPS:
        dfs.append(g.sample(n=CAP_COMPS[comp], random_state=42))
    else:
        dfs.append(g)

df_capped = pd.concat(dfs, ignore_index=True)

# 看一下新分布
comp_counts_after = (
    df_capped["competition"]
    .value_counts()
    .rename("num_pairs")
    .reset_index()
    .rename(columns={"index": "competition"})
)

print(comp_counts_after)


In [ ]:
combined_capped = df_capped.to_dict("records")


In [ ]:
import json

with open("/data/userdata/v-lijingyuan/vail_data/valid.json", "w", encoding="utf-8") as f:
    json.dump(combined_capped, f, indent=2, ensure_ascii=False)

print(f"Saved {len(combined_capped)} records")


In [ ]:
len(combined_capped)

In [ ]:
combined_capped[11]